In [26]:
# Cell 1: Imports
import speech_recognition as sr
import pyttsx3
import datetime
import requests
import json
import webbrowser
import wikipedia
import pywhatkit
import sqlite3
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [27]:
# Cell 2: 
try:
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
  
    engine.setProperty('voice', voices[1].id)
    print("TTS Engine Initialized Successfully.")
   
    engine.say("T.T.S. Engine Initialized.")
    engine.runAndWait()
except Exception as e:
    print(f"TTS Engine Error: {e}")


TTS Engine Initialized Successfully.


In [28]:
# Cell 3:

def speak(text):
    
    print(f"Assistant: {text}")
    try:
        engine.say(text)
        engine.runAndWait()
    except Exception as e:
        print(f"Speak Error: {e}")

def listen_command():
 
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("\nListening...")
        r.pause_threshold = 1
        r.adjust_for_ambient_noise(source, duration=1) 
        audio = r.listen(source)

    try:
        print("Recognizing...")
 
        command = r.recognize_google(audio, language='en-in') 
        print(f"User said: {command}\n")
    except sr.UnknownValueError:
        print("Sorry, I didn't get that. Please say that again.")
        return "None"
    except sr.RequestError:
        speak("Sorry, my speech service is down.")
        return "None"
    except Exception as e:
        print(e)
        return "None"
    return command.lower()

In [29]:
# Cell 4:


def get_weather(city_name):
  
    

    api_key = "9e45a7094d60a9de86901e08f0ea1acd" 
    
    if api_key == "YOUR_API_KEY":
        speak("Weather API key not set. Please get an API key from OpenWeatherMap website.")
        return

    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}&units=metric"
    
    try:
        response = requests.get(base_url)
        data = response.json()
        
        if data["cod"] != "404":
            main = data["main"]
            weather_desc = data["weather"][0]["description"]
            temp = main["temp"]
            report = f"The temperature in {city_name} is {temp}°C with {weather_desc}."
            speak(report)
        else:
            speak("Sorry, I couldn't find the city.")
    except requests.exceptions.RequestException:
        speak("Sorry, I couldn't connect to the weather service.")
    except Exception as e:
        speak("Sorry, I couldn't fetch the weather information right now.")


def setup_database():
 
    try:
        conn = sqlite3.connect('assistant_memory.db')
        cursor = conn.cursor()
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS notes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            note_text TEXT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        conn.commit()
        conn.close()
    except Exception as e:
        print(f"Database setup error: {e}")

def add_note(note):
 
    try:
        conn = sqlite3.connect('assistant_memory.db')
        cursor = conn.cursor()
        cursor.execute("INSERT INTO notes (note_text) VALUES (?)", (note,))
        conn.commit()
        conn.close()
        speak("I've remembered that.")
    except Exception as e:
        speak("Sorry, I couldn't remember that right now.")
        print(e)

def show_notes():
   
    try:
        conn = sqlite3.connect('assistant_memory.db')
        cursor = conn.cursor()
        cursor.execute("SELECT note_text FROM notes ORDER BY created_at DESC")
        all_notes = cursor.fetchall() 
        conn.close()
        
        if not all_notes:
            speak("You haven't told me to remember anything yet.")
        else:
            speak(f"You have {len(all_notes)} notes. Here they are:")
            for index, note in enumerate(all_notes):
                speak(f"Note {index + 1}: {note[0]}") 
    except Exception as e:
        speak("Sorry, I couldn't retrieve your notes right now.")
        print(e)

In [30]:


print("Training NLP model...")

training_data = [
    "hello", "hi", "hey", "how are you", "good morning", "what's up",
    "what's the time", "tell me the time", "current time", "time please",
    "what's the weather", "how is the weather", "temperature", "weather report",
    "open youtube", "go to youtube", "launch youtube",
    "open a website", "launch website", "open google",
    "search wikipedia", "who is", "what is", "tell me about",
    "play a song", "play music", "i want to listen to a song", "put on some music",
    "remember this", "make a note", "add a note", "remember",
    "show my notes", "what did I tell you to remember", "show all notes", "my notes",
    "goodbye", "exit", "stop", "see you later", "quit"
]


labels = [
    "greeting", "greeting", "greeting", "greeting", "greeting", "greeting",
    "get_time", "get_time", "get_time", "get_time",
    "get_weather", "get_weather", "get_weather", "get_weather",
    "open_youtube", "open_youtube", "open_youtube",
    "open_website", "open_website", "open_website",
    "search_wikipedia", "search_wikipedia", "search_wikipedia", "search_wikipedia",
    "play_song", "play_song", "play_song", "play_song",
    "remember_note", "remember_note", "remember_note", "remember_note",
    "show_notes", "show_notes", "show_notes", "show_notes",
    "exit", "exit", "exit", "exit", "exit"
]


vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(training_data)


model = LogisticRegression()
model.fit(X, labels)

print("NLP Model trained successfully!")

Training NLP model...
NLP Model trained successfully!


In [31]:
# Cell 6:

def run_assistant():
    speak("Hello! I am your personal assistant, powered by NLP made by soroj panda. How can I help you today?")
    
    setup_database() 
    
    while True:
        command = listen_command()
        
        if command == "None":
            continue

        try:
          
            command_vec = vectorizer.transform([command])
           
            intent = model.predict(command_vec)[0]
            
            print(f"[Debug] Detected Intent: {intent}") 

          
            
            if intent == 'greeting':
                speak("Hello! How are you?")
                
            elif intent == 'get_time':
                strTime = datetime.datetime.now().strftime("%I:%M %p") 
                speak(f"The current time is {strTime}")
                
            elif intent == 'get_weather':
                speak("Which city's weather do you want to know?")
                city = listen_command()
                if city != "None":
                    get_weather(city)
            
            elif intent == 'open_youtube':
                speak("Opening YouTube...")
                webbrowser.open("https://www.youtube.com")

            elif intent == 'open_website':
                speak("Which website should I open? Please say the full name, like google.com")
                site = listen_command()
                if site != "None":
                    site_url = site.replace(" ", "") 
                    speak(f"Opening {site_url}")
                    webbrowser.open(f"https://{site_url}")

            elif intent == 'search_wikipedia':
             
                query = command.replace("who is", "").replace("what is", "").replace("tell me about", "").replace("search wikipedia", "").strip()
                
                if not query: 
                    speak("What do you want to search on Wikipedia?")
                    query = listen_command()
                    
                if query != "None" and query:
                    try:
                        speak(f"Searching Wikipedia for {query}...")
                        results = wikipedia.summary(query, sentences=2)
                        speak("According to Wikipedia,")
                        speak(results)
                    except wikipedia.exceptions.PageError:
                        speak(f"Sorry, I couldn't find any Wikipedia page for {query}.")
                    except wikipedia.exceptions.DisambiguationError:
                        speak(f"'{query}' is ambiguous. Please be more specific.")
                    except Exception as e:
                        speak("Sorry, an error occurred while searching Wikipedia.")

            elif intent == 'play_song':
            
                song = command.replace("play song", "").replace("play music", "").replace("play", "").strip()

                if not song:
                    speak("What song or video should I play on YouTube?")
                    song = listen_command()
                
                if song != "None" and song:
                    speak(f"Playing {song} on YouTube...")
                    pywhatkit.playonyt(song)
            
            elif intent == 'remember_note':
                note_to_remember = command.replace("remember this", "").replace("remember", "").replace("make a note", "").replace("add a note", "").strip()

                if not note_to_remember:
                    speak("What should I remember?")
                    note_to_remember = listen_command()

                if note_to_remember != "None" and note_to_remember:
                    add_note(note_to_remember)
                else:
                    speak("I didn't catch that.")
            
            elif intent == 'show_notes':
                show_notes()

            elif intent == 'exit':
                speak("Goodbye! Have a great day.")
                break
                
            else:
              
                speak("Sorry, I don't understand that command yet. I am still learning.")
        
        except Exception as e:
            print(f"An error occurred in the main loop: {e}")
            speak("Sorry, something went wrong. Please try again.")

In [32]:
run_assistant()


Assistant: Hello! I am your personal assistant, powered by NLP made by soroj panda. How can I help you today?

Listening...
Recognizing...
Sorry, I didn't get that. Please say that again.

Listening...
Recognizing...
User said: weather

[Debug] Detected Intent: get_weather
Assistant: Which city's weather do you want to know?

Listening...
Recognizing...
User said: Kolkata

Assistant: The temperature in kolkata is 22.97°C with haze.

Listening...
Recognizing...
User said: open YouTube

[Debug] Detected Intent: open_youtube
Assistant: Opening YouTube...

Listening...
Recognizing...
Sorry, I didn't get that. Please say that again.

Listening...
Recognizing...
User said: exit

[Debug] Detected Intent: exit
Assistant: Goodbye! Have a great day.
